# Выбор локации для скважины

Вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).  
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.  
- Бюджет на разработку скважин в регионе — 10 млрд рублей.  
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.  
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

Импортируем все необходимые для дальнейшей работы библиотеки.

In [1]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import (train_test_split, 
                                     cross_val_score)
from sklearn.metrics import (mean_squared_error, 
                             r2_score)

Выгрузим данные о трех регионах, соответственно, в переменные data1, data2, data3

In [2]:
names = ['ПЕРВЫЙ', 'ВТОРОЙ', 'ТРЕТИЙ']

In [4]:
data1 = pd.read_csv('datasets/geo_data_0.csv')
data2 = pd.read_csv('datasets/geo_data_1.csv')
data3 = pd.read_csv('datasets/geo_data_2.csv')

files = [data1, data2, data3]

In [5]:
for x, y in zip(names, files):
    print(f'{x} регион: информация')
    display(y.head())
    print(y.info())
    print(y.describe(), end='\n\n\n\n\n')

ПЕРВЫЙ регион: информация


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.3437

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%         1.153055      -4.813172       2.011479      57.085625
75%         8.621015      -1.332816       3.999904     107.813044
max        29.421755      18.7340

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.174820       0.130359      59.450441
50%         0.009424      -0.009482       2.484236      94.925613
75%         1.158535       1.163678       4.858794     130.595027
max         7.238262       7.8448

- Пропусков нет, и кол-во признаков во всех трех колонках одинаковое. Значит, предобработка не требуется;  
- Второй регион имеет самые низкие показатели среднего и максимального значения в колонке с продуктом. Это может означать, что этот регион самый неприбыльный. Проверим это далее;

Теперь начинаем подготовку признаков.  

Для начала разделим датасеты на признаки и целевой признак, не включая колонку 'id' - она нам в обучении модели не поможет.

In [6]:
features1 = data1.drop(['id', 'product'], axis=1)
target1 = data1['product']

features2 = data2.drop(['id', 'product'], axis=1)
target2 = data2['product']

features3 = data3.drop(['id', 'product'], axis=1)
target3 = data3['product']

Теперь делим из на обучающую и валидационную выборки.

In [7]:
features_train1, features_valid1, target_train1, target_valid1 = (
    train_test_split(features1, target1, test_size=0.25)
)
features_train2, features_valid2, target_train2, target_valid2 = (
    train_test_split(features2, target2, test_size=0.25)
)
features_train3, features_valid3, target_train3, target_valid3 = (
    train_test_split(features3, target3, test_size=0.25)
)

print(features_train1.shape, features_valid1.shape)
print(features_train2.shape, features_valid2.shape)
print(features_train3.shape, features_valid3.shape)

(75000, 3) (25000, 3)
(75000, 3) (25000, 3)
(75000, 3) (25000, 3)


Теперь начинаем подготовку признаков.  

У нас будет регрессионная модель. Значит, преобразование категориальных моделей в численные нам не понадобится. Используем только масштабирование признаков.     

Стандартизировать данные мы будем функцией Standard Scaler. Для каждого региона (для каждой отдельной модели) будем масштабировать данные отдельно.

In [8]:
# сохраняем название колонок в отдельную переменную
columns = features1.columns

# создаем модель, которая будет масштабировать
scaler = StandardScaler()

# масштабируем первый регион
scaler.fit(features_train1)
scaler.transform(features_train1)
scaler.transform(features_valid1)

# масштабируем второй регион
scaler.fit(features_train2)
scaler.transform(features_train2)
scaler.transform(features_valid2)

# масштабируем третий регион
scaler.fit(features_train3)
scaler.transform(features_train3)
scaler.transform(features_valid3)

# проверяем
display(features_train1.head())
display(features_valid1.head())

,f0,f1,f2
10280,1.430998,-0.253906,0.313021
4687,1.821586,-0.112133,1.565152
68200,1.994459,0.530592,8.112532
87758,0.011347,0.039317,-0.705557
22241,0.004286,0.094829,-0.380777


,f0,f1,f2
17905,-0.252573,0.470926,5.216238
85165,-1.010592,0.276342,7.628469
83607,0.494880,-0.266659,1.075921
7161,0.086181,1.003273,-0.325700
2983,-0.458325,0.983127,0.319428


**С подготовкой признаков мы закончили:**

- Проверили данные на наличие пропусков. Пропуски не обнаружены;  
- Масштабировали данные для дальнейшего продуктивного обучения моделей;  
- Разделили данные на тренировочную и валидационную выборки;  

## Обучение и проверка модели

### Обучим модель и сделаем предсказания на валидационной выборке.

Так как в задаче есть условие использовать линейную регрессию, то модели будем обучать только на ней и не будем подбирать гиперпараметры для остальных регрессивных моделей.  

Обучим три модели на каждой их выборок.

In [9]:
# первая модель
model1 = LinearRegression()
model1.fit(features_train1, target_train1)
predicted_valid1 = model1.predict(features_valid1)

# вторая модель
model2 = LinearRegression()
model2.fit(features_train2, target_train2)
predicted_valid2 = model2.predict(features_valid2)

# третья модель
model3 = LinearRegression()
model3.fit(features_train3, target_train3)
predicted_valid3 = model3.predict(features_valid3)

Все предсказания и правильные ответы у нас сохранены в переменных predicted_valid и target_valid с соответствующим номером.

### Напечатаем на экране средний запас предсказанного сырья и RMSE модели.

Для первой модели:

In [10]:
model1_mean = predicted_valid1.mean()
model1_rmse = mean_squared_error(target_valid1, predicted_valid1, squared=False)

print('Среднее предсказанное значение: {:.2f}, rmse: {:.2f}'.format(model1_mean, model1_rmse))
print('Настоящее среднее: {:.2f}'.format(target_valid1.mean()))

Среднее предсказанное значение: 92.48, rmse: 37.63
Настоящее среднее: 92.54


Для второй модели:

In [11]:
model2_mean = predicted_valid2.mean()
model2_rmse = mean_squared_error(target_valid2, predicted_valid2, squared=False)

print('Среднее предсказанное значение: {:.2f}, rmse: {:.2f}'.format(model2_mean, model2_rmse))
print('Настоящее среднее: {:.2f}'.format(target_valid2.mean()))

Среднее предсказанное значение: 68.82, rmse: 0.89
Настоящее среднее: 68.81


Для третьей модели:

In [12]:
model3_mean = predicted_valid3.mean()
model3_rmse = mean_squared_error(target_valid3, predicted_valid3, squared=False)

print('Среднее предсказанное значение: {:.2f}, rmse: {:.2f}'.format(model3_mean, model3_rmse))
print('Настоящее среднее: {:.2f}'.format(target_valid3.mean()))

Среднее предсказанное значение: 94.59, rmse: 40.05
Настоящее среднее: 95.27


Лучшие предсказания, судя по маленькому отклонению, у второй модели. Давайте проверим еще раз наши модели теперь с помощью r2-метрики. У идеальной модели она должна стремиться к единице.

In [13]:
print(r2_score(target_valid1, predicted_valid1))
print(r2_score(target_valid2, predicted_valid2))
print(r2_score(target_valid3, predicted_valid3))

0.27878799187698977
0.9996245814479021
0.2007623355488889


### Проанализируем результаты

1) Вторая модель предсказывает точнее всех. Это выявил и низкий показатель rmse, и r2 крайне близкий к единице. Подозрительно идеально;  
2) У всех моделей и предсказнное, и истинное среднее практически не отличаются. Значит, для дальнейших рассчетов все модели нам все равно подойдут, так как мы будем считать общую прибыль по району добычи;  

## Подготовка к расчёту прибыли

### Введем ключевые значения для будущих рассчетов.

In [14]:
# кол-во скважин для исследования
RESEARCHING_WELLS = 500

# кол-во скважин для будущей добычи
EXTRACTING_WELLS = 200

# бюджет на один регион в млн рублей (10 млрд руб)
BUDGET_PER_REGION = 10000

# цена за 1 тыс баррелей нефти в млн рублей
PRICE_FOR_THOUSAND = 0.45

# вероятность убытков (2.5 %)
PROBABILITIES_OF_LOSSES = 0.025

### Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

По условию задачи бюджет может быть использован только на 200 скважин. Значит, для определения достаточного объема нам нужно разделить бюджет на кол-во возможных скважин.

In [15]:
budget_per_well = BUDGET_PER_REGION / EXTRACTING_WELLS
budget_per_well

50.0

Бюджет на одну скважину: 50 млн рублей. При такой сумме добыча в определенной скважине выйдет в ноль.  
Теперь посчитаем, какое это кол-во тыс. баррелей (потому что именно в тысячах у нас указана информация в датасете).

In [16]:
product_count = budget_per_well / PRICE_FOR_THOUSAND
product_count

111.11111111111111

### Выводы:

Необходимое кол-во продукта превышает среднее кол-во тыс баррелей по каждому отдельному региону. Значит, мы не можем бездумно выбрать случайные скважины во всех трех регионах, потому что нас может ждать неудача. Нужно выбрать регион с наименьшей вероятностью получить убытки.

## Расчёт прибыли и рисков 

Для рассчета потенциальной прибыли из одного региона нам нужно написать функцию, которая: 

1) На вход будет получать настоящие показатели, предсказания и максимально возможное кол-во скважин.  
2) Отсортирует предсказания и составит топ по прибыльности.  
3) На выходе будет выдавать сумму истинных значений, соответствующим топу по предсказаниям (в млн рублей)

In [17]:
def revenue(target, predicted, count):
    
    predicted = pd.Series(predicted)
    target = pd.Series(target).reset_index(drop=True)
    
    predicted = predicted.sort_values(ascending=False)
    indexes = predicted[:count].index
    total = target[indexes].sum()
    total = total * PRICE_FOR_THOUSAND
    
    return total

Функцию создали. Теперь посчитаем прибыль во всем трем районам и проверим работоспособность нашей функции

In [18]:
print(f'Прибыль для первого райнона: {revenue(target_valid1, predicted_valid1, EXTRACTING_WELLS)} млн рублей')
print(f'Прибыль для второго райнона: {revenue(target_valid2, predicted_valid2, EXTRACTING_WELLS)} млн рублей')
print(f'Прибыль для третьего райнона: {revenue(target_valid3, predicted_valid3, EXTRACTING_WELLS)} млн рублей')

Прибыль для первого райнона: 13249.9454247781 млн рублей
Прибыль для второго райнона: 12415.086696681512 млн рублей
Прибыль для третьего райнона: 12388.966948279061 млн рублей


Прибыль в самом лучшем случае могла бы составить более 10 млрд рублей в каждом регионе. Это при условии, что мы идеально случайным образом выберем из всех скважин - 200 самых прибыльныльных.  

Получается, каждый регион потенциально может окупить бюджет на разработку. Оценим риски по всем трем регионам.

Чтобы посчитать, окупится ли район с нужной нам вероятностью (2,5%), нужно:  

1) Взять 500 случайных значений скважин из региона  
2) Из этих 500 взять 200 с самой большой предсказанной выручкой  
3) Сложить выручку с этих самых прибыльных скважин    
5) Повторить таким образом 1000 раз   
6) Вычесть из выручки каждой выборки бюджет на разработку. Это будет чистая прибыль   
4) Оценить, убыточна она или нет  
7) Посчитать долю убыточных выборок от всего кол-ва и проверить, не больше ли это значение статистической значимости  
8) Если больше, то регион не окупится со слишком большой вероятностью, если меньше, то с достаточно низкой  

Сразу создадим фиксированный сид вероятности

In [19]:
state = np.random.RandomState(12345)

### Проверяем первый регион:

In [20]:
values = []
counter = 0

for i in range(1000):
    
    target_subsample = target_valid1.reset_index(drop=True).sample(n=RESEARCHING_WELLS, replace=True, random_state=state)
    pred_subsample = predicted_valid1[target_subsample.index]
    profit = revenue(target_subsample, pred_subsample, EXTRACTING_WELLS)
    profit -= BUDGET_PER_REGION
    values.append(profit)
    
    if profit < 0:
        counter += 1
        
result = counter / 1000
values = pd.Series(values)

lower = values.quantile(0.025)
upper = values.quantile(0.975)
print('Начало доверительного интервала: {:.2f}'.format(lower))
print('Конец доверительного интервала: {:.2f}'.format(upper))

if result < PROBABILITIES_OF_LOSSES:
    print('Первый регион подходит по критериям. Вероятность убытков = {:.1%}'.format(result))
else:
    print('Первый регион НЕ подходит по критериям. Вероятность убытков = {:.1%}'.format(result))
    
print(f'Средняя прибыль по выборкам: {pd.Series(values).mean()}, минимальная прибыль: {pd.Series(values).min()}')

Начало доверительного интервала: -91.40
Конец доверительного интервала: 949.25
Первый регион НЕ подходит по критериям. Вероятность убытков = 4.3%
Средняя прибыль по выборкам: 457.2110964956182, минимальная прибыль: -376.2130711239188


### Проверяем второй регион:

In [21]:
values = []
counter = 0

for i in range(1000):
    
    target_subsample = target_valid2.reset_index(drop=True).sample(n=RESEARCHING_WELLS, replace=True, random_state=state)
    pred_subsample = predicted_valid2[target_subsample.index]
    profit = revenue(target_subsample, pred_subsample, EXTRACTING_WELLS)
    profit -= BUDGET_PER_REGION
    values.append(profit)
    
    if profit < 0:
        counter += 1
        
result = counter / 1000
values = pd.Series(values)

lower = values.quantile(0.025)
upper = values.quantile(0.975)
print('Начало доверительного интервала: {:.2f}'.format(lower))
print('Конец доверительного интервала: {:.2f}'.format(upper))

if result < PROBABILITIES_OF_LOSSES:
    print('Второй регион подходит по критериям. Вероятность убытков = {:.1%}'.format(result))
else:
    print('Второй регион НЕ подходит по критериям. Вероятность убытков = {:.1%}'.format(result))
    
print(f'Средняя прибыль по выборкам: {pd.Series(values).mean()}, минимальная прибыль: {pd.Series(values).min()}')

Начало доверительного интервала: 52.47
Конец доверительного интервала: 860.23
Второй регион подходит по критериям. Вероятность убытков = 1.7%
Средняя прибыль по выборкам: 444.88546052227264, минимальная прибыль: -250.05175411713572


### Проверяем третий регион:

In [22]:
values = []
counter = 0

for i in range(1000):
    
    target_subsample = target_valid3.reset_index(drop=True).sample(n=RESEARCHING_WELLS, replace=True, random_state=state)
    pred_subsample = predicted_valid3[target_subsample.index]
    profit = revenue(target_subsample, pred_subsample, EXTRACTING_WELLS)
    profit -= BUDGET_PER_REGION
    values.append(profit)
    
    if profit < 0:
        counter += 1
        
result = counter / 1000
values = pd.Series(values)

lower = values.quantile(0.025)
upper = values.quantile(0.975)
print('Начало доверительного интервала: {:.2f}'.format(lower))
print('Конец доверительного интервала: {:.2f}'.format(upper))

if result < PROBABILITIES_OF_LOSSES:
    print('Третий регион подходит по критериям. Вероятность убытков = {:.1%}'.format(result))
else:
    print('Третий регион НЕ подходит по критериям. Вероятность убытков = {:.1%}'.format(result))
    
print(f'Средняя прибыль по выборкам: {pd.Series(values).mean()}, минимальная прибыль: {pd.Series(values).min()}')

Начало доверительного интервала: -88.82
Конец доверительного интервала: 974.48
Третий регион НЕ подходит по критериям. Вероятность убытков = 5.0%
Средняя прибыль по выборкам: 426.54929986656725, минимальная прибыль: -340.93389234500137


### Выводы:

По результатам нашего исследования, могу рекомендовать второй регион для добычи с максимальной прибылью.  

- Только второй регион проходит по условию вероятности получения убытков (не больше 2,5%). Остальные имеют слишком высокий риск убытков, поэтому они нам не подходят.  
- Отрицательная минимальная прибыль есть во всех трех регионах.  
- Нижняя граница доверительного интервала положительная только во втором регионе. Это доказывает, что с заданной вероятностью только второй регион не понесет убытки.  
- Средняя выручка по выборкам самая высокая во втором регионе. Это также доказывает, что необходимо остановиться на нем.  